In [28]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat_model = ChatOpenAI(model="gpt-4-1106-preview")

In [29]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

llm.invoke(text)

'\n\nGlee Socks.'

In [30]:
chat_model.invoke(messages)

AIMessage(content="Choosing a company name for a business that specializes in colorful socks can be quite fun. You'll want a name that is catchy, memorable, and reflects the vibrant and playful nature of your product. Here are some suggestions:\n\n1. Sock Spectrum\n2. VividFeet\n3. HueHosiery\n4. ColorPopSocks\n5. RainbowToes\n6. PizzazzSocks\n7. BrightSteps\n8. KaleidoSox\n9. PrismPeds\n10. ArtiSox\n11. DazzleHeels\n12. Socktastic Colors\n13. FootFiesta\n14. ZingSocks\n15. HappyHeel Threads\n\nRemember to check for domain availability if you plan to sell online, and ensure the name isn't already trademarked or in heavy use by another company in your market. It's always a good idea to test your favorite names with potential customers to see which one resonates the most.")

In [31]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [32]:
from langchain.prompts.chat import ChatPromptTemplate

In [33]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

msgs = chat_prompt.format_messages(
    input_language="english",
    output_language="afrikaans",
    text="Hello there, you fool."
)

msgs

[SystemMessage(content='You are a helpful assistant that translates english to afrikaans.'),
 HumanMessage(content='Hello there, you fool.')]

In [34]:
chat_model.invoke(msgs)

AIMessage(content='Hallo daar, jy dwaas.')

In [35]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
       return text.strip().split(", ")     

CommaSeparatedListOutputParser().parse("hy, bye")

['hy', 'bye']

In [50]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):

    def parse(self, text:str) -> List[str]:
        return text.strip().split(", ")

template = """
You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more.
"""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)

chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})

['red', 'blue', 'green', 'yellow', 'purple']

In [51]:
chat_prompt

ChatPromptTemplate(input_variables=['text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nYou are a helpful assistant who generates comma separated lists.\nA user will pass in a category, and you should generate 5 objects in that category in a comma separated list.\nONLY return a comma separated list, and nothing more.\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [62]:
# CustomPrompt
import inspect
from langchain.prompts import StringPromptTemplate
from pydantic.v1 import BaseModel, validator

PROMPT = """\
Given the function name and source code, generate an simplified English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """Custom prompt template"""

    @validator("input_variables")
    def validate_input_variables(cls, v):  # Note: This is awesome -- it's a pre-flight error check
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input variable.")
        return v

    def format(self, **kwargs) -> str:
        source_code = inspect.getsource(kwargs["function_name"])

        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self):
        return "function-explainer"

In [63]:
fn_explainer = FunctionExplainerPromptTemplate(
    input_variables=["function_name"]
)

fn_explainer.format(function_name=inspect.signature)

chain = fn_explainer | ChatOpenAI()
chain.invoke({"function_name": inspect.signature})

AIMessage(content='The function "signature" takes in an object and returns a signature object for that object. It has the option to follow wrapped objects, use specific global and local variables, and evaluate strings. The function is implemented by calling the "Signature.from_callable" function with the given parameters.')

In [77]:
# Let's figure out how to do output parsing

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo")

class Joke(BaseModel):  # The output model
    setup: str = Field(description="question to setup the joke")
    punchline: str = Field(description="answer to resolve the joke")

parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(  # The prompt fed into as input
    template="Answer the query.\n{format_instructions}\n{query}\n",  # This is the prompt template
    input_variables=["query"],  # These are the only inputs we need to provide.
    partial_variables={
        "format_instructions": parser.get_format_instructions()  # We specify particular format instructions that get put into the template
    }
)

chain = prompt | model | parser
output = chain.invoke({"query": "Tell me a joke."})

In [78]:
output

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')